In [203]:
push!(LOAD_PATH,"../WebTools/src/")
using WebTools

using DataFrames,Dates,Distributions

In [204]:
elements=[l1*l2 for l1 in 'A':'Z' for l2 in 'A':'Z']

products="Product_".*[l1*l2 for l1 in 'A':'Z' for l2 in 'A':'Z']
clients="Client_".*[string(l1) for l1 in 'A':'Z']

product_price=Dict(p=>rand(0.1:0.1:100) for p in products)
client_disc=Dict(c=>rand([0.0,0.1,0.2,0.3,0.4]) for c in clients)
client_fraud=Dict(c=>rand()>0.95 ? true : false for c in clients);

In [178]:
nr=1000000
invoices=DataFrame()
invoices[:Date]=rand(Date(2019,1,1):Dates.Day(1):Date(2019,1,31),nr)
invoices[:Client]=rand(clients,nr)
invoices[:Product]=rand(products,nr)
invoices[:Qty]=rand(10:100,nr)
invoices[:UnitPriceGr]=map(x->product_price[x],invoices[:Product])
invoices[:Discount]=map(x->client_disc[x],invoices[:Client])
invoices[:UnitPriceReal]=invoices[:UnitPriceGr].*(1 .-invoices[:Discount])
invoices[:Noise]=[rand()>0.95 ? rand(LogNormal(0.01,0.025)) : 1.0 for r in 1:nr]
invoices[:Fraud]=map(x->client_fraud[x] ? rand(0.95:0.01:0.99) : 1.0,invoices[:Client])
invoices[:UnitPrice]=invoices[:UnitPriceReal].*invoices[:Noise].*invoices[:Fraud]
invoices[:Value]=invoices[:Qty].*invoices[:UnitPrice]

invoices

,Date,Client,Product,Qty,UnitPriceGr,Discount,UnitPriceReal,Noise,Fraud,UnitPrice,Value
,Date,String,String,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2019-01-06,Client_Z,Product_AI,42,14.4,0.0,14.4,1.0,1.0,14.4,604.8
2,2019-01-11,Client_S,Product_KC,23,27.1,0.2,21.68,1.0,1.0,21.68,498.64
3,2019-01-17,Client_D,Product_HA,73,13.9,0.0,13.9,1.0,1.0,13.9,1014.7
4,2019-01-13,Client_K,Product_XC,77,24.8,0.1,22.32,1.0,1.0,22.32,1718.64
5,2019-01-28,Client_P,Product_VG,90,71.5,0.1,64.35,0.988495,1.0,63.6097,5724.87
6,2019-01-19,Client_N,Product_LO,83,14.6,0.2,11.68,1.0,1.0,11.68,969.44
7,2019-01-25,Client_S,Product_LN,65,46.9,0.2,37.52,1.0,1.0,37.52,2438.8
8,2019-01-03,Client_B,Product_RE,61,11.1,0.2,8.88,1.0,1.0,8.88,541.68
9,2019-01-06,Client_C,Product_VP,11,81.1,0.4,48.66,1.0,1.0,48.66,535.26


In [179]:
summary=by(invoices,[:Client,:Product],df->DataFrame(Value=sum(df[:Value]),meanPrice=mean(df[:UnitPrice]),stdPrice=std(df[:UnitPrice])))
summary[:cvPrice]=summary[:stdPrice]./summary[:meanPrice]
summary[:Risk]=summary[:cvPrice].*summary[:Value];

In [201]:
opt=Dict("xAxis"=>Dict("type"=>"value"),"yAxis"=>Dict("type"=>"value"),
            "series"=>[])
push!(opt["series"],Dict("type"=>"scatter","data"=>map(x->Dict("value"=>[round(x[:cvPrice],digits=5),x[:Value],x[:Risk],x[:Client],x[:Product]]),eachrow(summary))))
opt["visualMap"]=Dict("dimension"=>2,"min"=>minimum(summary[:Risk]),"max"=>maximum(summary[:Risk]),"color"=>["#f01104","#2f8946"])
opt["title"]=Dict("text"=>"Risco Preço nas Vendas por Cliente/Produto","left"=>"center")
ec=EChart(opt,width=1200,height=800);

In [200]:
open("sales_risk.html", "w") do f
    write(f,"""<!DOCTYPE html>
<html>
<head>
    <title>Demo</title>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <script src="https://cdnjs.cloudflare.com/ajax/libs/echarts/4.3.0/echarts-en.min.js"></script>
</head>""")
    write(f,"<body>")
    write(f,html(ec))
    write(f,"</body>")
    write(f,"""<script>myChart.setOption({"tooltip":{"padding":10,"borderColor":"#777","borderWidth":1,"backgroundColor":"#242","formatter":
                    function (params, ticket, callback) {
                        return "Cliente : "+params.value[3]+"<br>Produto : "+params.value[4]+"<br>Vendas : "+Intl.NumberFormat().format(params.value[1])+"<br>Coef Var Preço : "+params.value[0]
                        }
                    }},false,true);</script>
    """)                
    
end


433